# KD Tree for Bottleneck TSP

In [1]:
%load_ext autoreload
%autoreload 2

## Load Spotify Data

In [2]:
import tekore as tk
from smoothify.features import construct_features
import numpy as np

In [3]:
ACCESS_TOKEN = "BQAyJmwrlYm3e5i60Ry4Kg_8rO8ONKInbt1ETax0Z3XmhCNYL_IrzWstCJhw3jZD9l8V5zTNvKyPx9jpBlYLT99-YdWZwtRAZyWKS3wbKv4cQSeSR-hTNGsgUCkR3E6-wCIvCQqm2PGo8Wx-kLBDKxhuJX-s1EZK2LOpK7zvXIE"
spotify = tk.Spotify(ACCESS_TOKEN, asynchronous=True, max_limits_on=True, chunked_on=True)

### Fetch current user's tracks

In [4]:
saved_tracks_page = await spotify.saved_tracks()
track_list = [track.track async for track in spotify.all_items(saved_tracks_page)]

### Query Spotify API for audio features

In [5]:
# Extract track IDs so we can fetch additional info
track_id_list = [track.id for track in track_list]
# Get track audio features
track_features_list = await spotify.tracks_audio_features(track_id_list)

### Construct & normalize audio features

In [6]:
features_df = construct_features(audio_features_list=track_features_list)
features_df.head(3)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-0.335216,0.848965,0.075837,-0.717004,-0.440660,-0.792444,-0.753034,0.785131,1.438342
1,1.200938,1.397294,1.086885,2.753551,-0.153943,-0.624321,0.997408,0.266301,-0.030423
2,0.946060,0.234172,0.165459,-0.358500,-0.707600,1.622556,-0.617096,1.523465,0.216439


In [7]:
points = np.array(features_df)

## Compute the best path through the points

In [8]:
import random
from functools import partial
from typing import List, Tuple

import pandas as pd
import scipy.spatial
from scipy.spatial import cKDTree
from joblib import Parallel, delayed

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
sns.set_theme()

In [9]:
from smoothify.optim.bottleneck_tsp import KDTreeBottleneckTSP

In [10]:
optimizer = KDTreeBottleneckTSP(points=points)
results = optimizer.get_best_path()
best_path = results.best_path
max_dist = results.min_max_dist
print(f"Max edge length: {max_dist}")

  0%|                                                                                                                                                                               | 0/492 [00:00<?, ?it/s]

[ProgressParallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 492/492 [00:29<00:00, 16.68it/s]

[ProgressParallel(n_jobs=-1)]: Done 492 out of 492 | elapsed:   29.5s finished
Max edge length: 5.138054353091741


In [11]:
for node_idx in best_path:
    print(track_list[node_idx].uri)

spotify:track:75zVWxQHkFkNKBVVQGwwyx
spotify:track:6GWNh4A7t4UHtUupgL5Evj
spotify:track:6G8f5H3FkCKIDaC5o2ugu9
spotify:track:0o8PAzsSLDa4MVQJixGVty
spotify:track:2PtBhfoPZ6VYtXkrE5FrCH
spotify:track:2tpWsVSb9UEmDRxAl1zhX1
spotify:track:2lZHCWCUkxLuQ5skJhkoyS
spotify:track:6U7M7qK0pOnifpA2T3GozN
spotify:track:1kyvcBW8iHkwSaDAL6FfxB
spotify:track:7aeYPexgesMMIFCzakAaV3
spotify:track:5GayeBzzXIEl6fY0cP42Z1
spotify:track:4IWLawtdMznqdOnbOR9cWW
spotify:track:37aStseEdbd33KlmUlIHX6
spotify:track:2TsMgZTaacXikcOAEezzyk
spotify:track:7HC3sppue0re7HPfo5zscF
spotify:track:6g1NlCpW7fgqDnWbCCDrHl
spotify:track:3OwPSJu609AMzotCEyoMiO
spotify:track:5tf1VVWniHgryyumXyJM7w
spotify:track:3HXMtAq0jJSGFTxzOYly8X
spotify:track:5pQDgXHURWaMrfxsmxUxiI
spotify:track:37qQL8k0K15vUdkUNfKC4V
spotify:track:5or3DfWwQPvoFSEKhifZaq
spotify:track:2V65y3PX4DkRhy1djlxd9p
spotify:track:1Thv8uCYzyOFC7PME9J936
spotify:track:7F5NU5LwUjAchegDbvmNdE
spotify:track:2NXRJXATLZb5bk1xIrODvX
spotify:track:1EsdqTsiQPaUJ82iy7KfS1
s